# Help functions

In [123]:
import imageio
import numpy as np

def LoadImage(filename):
    im = imageio.imread(filename)
    im = im[:, :, 0]
    im = im.astype('float32')
    im  = im/255
    return im

# Parameters

In [124]:
image_size = 28

# Locate data and store to datasetFIles

In [125]:
from os import listdir
from os.path import isfile, join

dirs = listdir("Data")
datasetFiles = {}

In [126]:
for dir in dirs:
    filesInCurrentFolder = listdir("Data/" + dir)
    datasetFiles[dir] = []
    for file in filesInCurrentFolder:
        if file.endswith(".png"):
            datasetFiles[dir].append("Data/" + dir + "/" + file)

# Load and process data

In [141]:
import imageio
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import np_utils 

dataset = []
labelsNames = {}
labels = []
labelsNamesList = []

currentLabelIndex = 0

for tag in datasetFiles.keys():
    labelsNames[currentLabelIndex] = tag
    labelsNamesList.append(tag)
    for file in datasetFiles[tag]:
        im = LoadImage(file)
        dataset.append(im)
        labels.append(currentLabelIndex)
        
    currentLabelIndex+=1
        
dataset = np.array(dataset)
dataset = dataset.reshape(dataset.shape[0], image_size, image_size, 1)
labels = np.array(labels)
labels = np_utils.to_categorical(labels)

x_train, x_test, y_train, y_test = train_test_split(dataset, labels, test_size=0.33, random_state=42)

class_count = len(labelsNames.keys())

C:\Users\aaa\AppData\Local\Temp/ipykernel_19964/3791481005.py:5: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  im = imageio.imread(filename)


# Model (OLD)

In [128]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,Flatten,Dropout,Dense,MaxPooling2D
from tensorflow.keras.optimizers import SGD
from keras.utils import np_utils 

model = Sequential()

model.add(Conv2D(32, kernel_size=(3,3), activation='relu',input_shape=(image_size,image_size,1), padding='SAME'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64,(3,3),activation='relu',padding='SAME'))
model.add(Dense(64, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())

model.add(Dense(128, activation='relu'))

model.add(Dense(50, activation='relu'))
model.add(Dense(class_count, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=SGD(0.01), metrics=['accuracy'])

In [107]:
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 28, 28, 32)        320       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 14, 14, 64)        18496     
                                                                 
 dense_7 (Dense)             (None, 14, 14, 64)        4160      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 7, 7, 64)         0         
 2D)                                                             
                                                                 
 dropout_2 (Dropout)         (None, 7, 7, 64)         

# Leaning (OLD)

In [108]:
batch_size=32
epochs=3

plotting_data = model.fit(x_train,
                          y_train,
                          batch_size=batch_size,
                          epochs=epochs,
                          verbose=1,
                          validation_data=(x_test, y_test))

loss,accuracy = model.evaluate(x_test, y_test,verbose=0)

print('Test loss ---> ',str(round(loss*100,2)) +str('%'))
print('Test accuracy ---> ',str(round(accuracy*100,2)) +str('%'))

Epoch 1/3
1/1 [==============================] - 1s 601ms/step - loss: 0.6889 - accuracy: 0.5000 - val_loss: 0.6871 - val_accuracy: 0.5714
Epoch 2/3
1/1 [==============================] - 0s 45ms/step - loss: 0.6873 - accuracy: 0.5357 - val_loss: 0.6865 - val_accuracy: 0.5714
Epoch 3/3
1/1 [==============================] - 0s 44ms/step - loss: 0.6861 - accuracy: 0.6786 - val_loss: 0.6857 - val_accuracy: 0.5714
Test loss --->  68.57%
Test accuracy --->  57.14%


# Model 2

In [129]:
import tensorflow as tf
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Convolution2D(
    input_shape=(image_size,image_size,1),
    kernel_size=5,
    filters=32,
    padding='same',
    activation=tf.keras.activations.relu
))
model.add(tf.keras.layers.MaxPooling2D(
    pool_size=2,
    strides=2
))

model.add(tf.keras.layers.Convolution2D(
    kernel_size=3,
    filters=32,
    padding='same',
    activation=tf.keras.activations.relu,
))
model.add(tf.keras.layers.MaxPooling2D(
    pool_size=2,
    strides=2
))

model.add(tf.keras.layers.Convolution2D(
    kernel_size=3,
    filters=64,
    padding='same',
    activation=tf.keras.activations.relu
))
model.add(tf.keras.layers.MaxPooling2D(
    pool_size=2,
    strides=2
))

model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(
    units=512,
    activation=tf.keras.activations.relu
))

model.add(tf.keras.layers.Dense(
    units=class_count,
    activation=tf.keras.activations.softmax
))

adam_optimizer = tf.keras.optimizers.Adam(learning_rate=0.003)
rms_prop_optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001)
sgd_optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

model.compile(
    optimizer=adam_optimizer,
    loss=tf.keras.losses.categorical_crossentropy,
    metrics=['accuracy']
)

# Learning 2

In [130]:
# Dataset batching
batch_size = 2000
prefetch_buffer_batches = 10

# Training
epochs = 40
steps_per_epoch = 500




plotting_data = model.fit(x_train,
                          y_train,
                          batch_size=batch_size,
                          epochs=epochs,
                          verbose=1,
                          validation_data=(x_test, y_test))

loss,accuracy = model.evaluate(x_test, y_test,verbose=0)

print('Test loss ---> ',str(round(loss*100,2)) +str('%'))
print('Test accuracy ---> ',str(round(accuracy*100,2)) +str('%'))

Epoch 1/40
1/1 [==============================] - 1s 595ms/step - loss: 1.6055 - accuracy: 0.1233 - val_loss: 1.5685 - val_accuracy: 0.2778
Epoch 2/40
1/1 [==============================] - 0s 68ms/step - loss: 1.4663 - accuracy: 0.3151 - val_loss: 1.4929 - val_accuracy: 0.3056
Epoch 3/40
1/1 [==============================] - 0s 64ms/step - loss: 1.2995 - accuracy: 0.3836 - val_loss: 1.2349 - val_accuracy: 0.4444
Epoch 4/40
1/1 [==============================] - 0s 66ms/step - loss: 1.0276 - accuracy: 0.6575 - val_loss: 0.9140 - val_accuracy: 0.6111
Epoch 5/40
1/1 [==============================] - 0s 67ms/step - loss: 0.7125 - accuracy: 0.8082 - val_loss: 0.5453 - val_accuracy: 0.8056
Epoch 6/40
1/1 [==============================] - 0s 67ms/step - loss: 0.3941 - accuracy: 0.9315 - val_loss: 0.2998 - val_accuracy: 1.0000
Epoch 7/40
1/1 [==============================] - 0s 66ms/step - loss: 0.2139 - accuracy: 0.9589 - val_loss: 0.7781 - val_accuracy: 0.7778
Epoch 8/40
1/1 [==========

# Save model

In [143]:
model.save("Model.h5")

# Convert
import tf2onnx
import onnxruntime as rt

spec = (tf.TensorSpec((None, image_size, image_size, 1), tf.float32, name="input"),)
output_path = "Model.onnx"

model_proto, _ = tf2onnx.convert.from_keras(model, input_signature=spec, opset=13, output_path=output_path)
output_names = [n.name for n in model_proto.graph.output]

# Class names
import json
labels_json = {}
labels_json["Items"] = labelsNamesList
json_object = json.dumps(labels_json, indent = 4)
f = open("ClassNames.json", "w")
f.write(json_object)
f.close()